# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [3]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [4]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [6]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [8]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [9]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

In [10]:
# MY SOLUTION:

# Import additional functions we'll need
from pyspark.sql.functions import col, count, countDistinct, sum as spark_sum, desc

In [11]:
# Data Exploration and Cleaning
print("Citations DataFrame info:")
citations.printSchema()
print(f"Total citations: {citations.count()}")

print("\nPatents DataFrame info:")
patents.printSchema()
print(f"Total patents: {patents.count()}")

# Check for null values in key columns
print(f"\nNull values in citations CITING: {citations.filter(col('CITING').isNull()).count()}")
print(f"Null values in citations CITED: {citations.filter(col('CITED').isNull()).count()}")
print(f"Null values in patents PATENT: {patents.filter(col('PATENT').isNull()).count()}")
print(f"Null values in patents POSTATE: {patents.filter(col('POSTATE').isNull()).count()}")

Citations DataFrame info:
root
 |-- CITING: integer (nullable = true)
 |-- CITED: integer (nullable = true)

Total citations: 16522438

Patents DataFrame info:
root
 |-- PATENT: integer (nullable = true)
 |-- GYEAR: integer (nullable = true)
 |-- GDATE: integer (nullable = true)
 |-- APPYEAR: integer (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- POSTATE: string (nullable = true)
 |-- ASSIGNEE: integer (nullable = true)
 |-- ASSCODE: integer (nullable = true)
 |-- CLAIMS: integer (nullable = true)
 |-- NCLASS: integer (nullable = true)
 |-- CAT: integer (nullable = true)
 |-- SUBCAT: integer (nullable = true)
 |-- CMADE: integer (nullable = true)
 |-- CRECEIVE: integer (nullable = true)
 |-- RATIOCIT: double (nullable = true)
 |-- GENERAL: double (nullable = true)
 |-- ORIGINAL: double (nullable = true)
 |-- FWDAPLAG: double (nullable = true)
 |-- BCKGTLAG: double (nullable = true)
 |-- SELFCTUB: double (nullable = true)
 |-- SELFCTLB: double (nullable = true)
 |-- SECDU

In [12]:
# Filter patents to only include those with state information (US patents)
us_patents = patents.filter(col('POSTATE').isNotNull() & (col('COUNTRY') == 'US'))
print(f"US patents with state information: {us_patents.count()}")

# Cache this since we'll use it multiple times
us_patents.cache()

US patents with state information: 1784989


DataFrame[PATENT: int, GYEAR: int, GDATE: int, APPYEAR: int, COUNTRY: string, POSTATE: string, ASSIGNEE: int, ASSCODE: int, CLAIMS: int, NCLASS: int, CAT: int, SUBCAT: int, CMADE: int, CRECEIVE: int, RATIOCIT: double, GENERAL: double, ORIGINAL: double, FWDAPLAG: double, BCKGTLAG: double, SELFCTUB: double, SELFCTLB: double, SECDUPBD: double, SECDLWBD: double]

In [13]:
# Create the intermediate join table
# First, join citations with patents to get the cited patent's state
citations_with_cited_state = citations.join(
    us_patents.select('PATENT', 'POSTATE').withColumnRenamed('POSTATE', 'Cited_State'),
    citations['CITED'] == us_patents['PATENT'],
    'inner'
).select('CITING', 'CITED', 'Cited_State')

print(f"Citations with cited state info: {citations_with_cited_state.count()}")

# Cache this intermediate result
citations_with_cited_state.cache()

Citations with cited state info: 9259246


DataFrame[CITING: int, CITED: int, Cited_State: string]

In [14]:
# Join with citing patent's state information
full_citation_data = citations_with_cited_state.join(
    us_patents.select('PATENT', 'POSTATE').withColumnRenamed('POSTATE', 'Citing_State'),
    citations_with_cited_state['CITING'] == us_patents['PATENT'],
    'inner'
).select('CITED', 'Cited_State', 'CITING', 'Citing_State')

print(f"Full citation data with both states: {full_citation_data.count()}")

# Cache this important intermediate result
full_citation_data.cache()

# Show sample of the intermediate result
print("\nSample of intermediate citation data:")
full_citation_data.show(10)

Full citation data with both states: 6920796

Sample of intermediate citation data:
+-------+-----------+-------+------------+
|  CITED|Cited_State| CITING|Citing_State|
+-------+-----------+-------+------------+
|3815160|         NY|3858597|          MT|
|3675252|         AZ|3858597|          MT|
|3741706|         OH|3859029|          NY|
|3685936|         OH|3859029|          NY|
|3368197|         MI|3859627|          MI|
|3359539|         CA|3859627|          MI|
|3729059|         NY|3859627|          MI|
|3381268|         DC|3859627|          MI|
|3340523|         FL|3859627|          MI|
|3430989|         CA|3860038|          AZ|
+-------+-----------+-------+------------+
only showing top 10 rows



In [15]:
# Filter for same-state citations and count them per patent
same_state_citations = full_citation_data.filter(
    col('Cited_State') == col('Citing_State')
)

print(f"Same-state citations: {same_state_citations.count()}")

# Count same-state citations per citing patent
same_state_counts = same_state_citations.groupBy('CITING', 'Citing_State').agg(
    count('CITED').alias('same_state_count')
)

print(f"Patents with same-state citations: {same_state_counts.count()}")
same_state_counts.cache()

Same-state citations: 1488330
Patents with same-state citations: 571919


DataFrame[CITING: int, Citing_State: string, same_state_count: bigint]

In [16]:
# Join back with original patent data to get all columns
# First, let's see what columns we need from the patents table
print("Patent columns:")
patents.columns

Patent columns:


['PATENT',
 'GYEAR',
 'GDATE',
 'APPYEAR',
 'COUNTRY',
 'POSTATE',
 'ASSIGNEE',
 'ASSCODE',
 'CLAIMS',
 'NCLASS',
 'CAT',
 'SUBCAT',
 'CMADE',
 'CRECEIVE',
 'RATIOCIT',
 'GENERAL',
 'ORIGINAL',
 'FWDAPLAG',
 'BCKGTLAG',
 'SELFCTUB',
 'SELFCTLB',
 'SECDUPBD',
 'SECDLWBD']

In [17]:
# Create the final result with all patent information
final_result = us_patents.join(
    same_state_counts,
    us_patents['PATENT'] == same_state_counts['CITING'],
    'left'
).select(
    us_patents['PATENT'],
    us_patents['GYEAR'],
    us_patents['GDATE'],
    us_patents['APPYEAR'],
    us_patents['COUNTRY'],
    us_patents['POSTATE'],
    us_patents['ASSIGNEE'],
    us_patents['ASSCODE'],
    us_patents['CLAIMS'],
    us_patents['NCLASS'],
    us_patents['CAT'],
    us_patents['SUBCAT'],
    us_patents['CMADE'],
    us_patents['CRECEIVE'],
    us_patents['RATIOCIT'],
    us_patents['GENERAL'],
    us_patents['ORIGINAL'],
    us_patents['FWDAPLAG'],
    us_patents['BCKGTLAG'],
    us_patents['SELFCTUB'],
    us_patents['SELFCTLB'],
    us_patents['SECDUPBD'],
    us_patents['SECDLWBD'],
    col('same_state_count')
)

# Replace null same_state_count with 0 for patents that don't cite any same-state patents
final_result = final_result.fillna({'same_state_count': 0})

In [18]:
# Get top 10 patents with most same-state citations
top_10_same_state = final_result.filter(
    col('same_state_count') > 0
).orderBy(
    desc('same_state_count')
).limit(10)

print("Top 10 patents with most same-state citations:")
top_10_same_state.show()

Top 10 patents with most same-state citations:
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|same_state_count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  NULL|   326|  4|    46|  159|       0|     1.0|   NULL|  0.6186|    NULL|  4.8868|  0.0455|   0.044|    NULL|    NULL|             125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  NULL|   114|  5|    55|  200|       0|   0.995|   NULL|  0.7201|    NULL|   12.45|     0.

In [19]:
# Verify our results by checking one of the top patents
print("\nVerifying results for the top patent:")
top_patent = top_10_same_state.first()
top_patent_id = top_patent['PATENT']
top_patent_state = top_patent['POSTATE']

print(f"Top patent: {top_patent_id} from {top_patent_state}")

# Get all citations made by this patent
top_patent_citations = full_citation_data.filter(
    col('CITING') == top_patent_id
)

print(f"Total citations by this patent: {top_patent_citations.count()}")

same_state_for_top = top_patent_citations.filter(
    col('Citing_State') == col('Cited_State')
)

print(f"Same-state citations: {same_state_for_top.count()}")
print("Sample same-state citations:")
same_state_for_top.show(10)


Verifying results for the top patent:
Top patent: 5959466 from CA
Total citations by this patent: 144
Same-state citations: 125
Sample same-state citations:
+-------+-----------+-------+------------+
|  CITED|Cited_State| CITING|Citing_State|
+-------+-----------+-------+------------+
|5349249|         CA|5959466|          CA|
|5600264|         CA|5959466|          CA|
|5357153|         CA|5959466|          CA|
|5600263|         CA|5959466|          CA|
|4527115|         CA|5959466|          CA|
|5598108|         CA|5959466|          CA|
|5068603|         CA|5959466|          CA|
|5530378|         CA|5959466|          CA|
|5631577|         CA|5959466|          CA|
|5448493|         CA|5959466|          CA|
+-------+-----------+-------+------------+
only showing top 10 rows



In [20]:
# Clean up - unpersist cached DataFrames
us_patents.unpersist()
citations_with_cited_state.unpersist()
full_citation_data.unpersist()
same_state_counts.unpersist()

print("Solution implemented successfully!")

Solution implemented successfully!
